In [1]:
import pandas as pd

In [2]:
TFL = pd.read_csv('tfl_journeys_final.csv')

In [3]:
%%sql
SELECT *, COUNT(*)
FROM TFL
GROUP BY MONTH, YEAR, DAYS, REPORT_DATE, JOURNEY_TYPE, JOURNEYS_MILLIONS
HAVING COUNT(*) > 1;

,month,year,days,report_date,journey_type,journeys_millions,count_star()


In [6]:
%%sql
SELECT *
FROM TFL
WHERE MONTH IS NULL OR YEAR IS NULL OR DAYS IS NULL OR REPORT_DATE IS NULL OR JOURNEY_TYPE IS NULL OR JOURNEYS_MILLIONS IS NULL
LIMIT 5;

,month,year,days,report_date,journey_type,journeys_millions
0,1,2010,31,06/30/2010,Overground,NaN
1,2,2010,28,06/30/2010,Overground,NaN
2,3,2010,31,06/30/2010,Overground,NaN
3,4,2010,30,09/30/2010,Overground,NaN
4,5,2010,31,09/30/2010,Overground,NaN


In [9]:
%%sql
CREATE TABLE CLEAN_TFL AS
SELECT *
FROM TFL
WHERE JOURNEYS_MILLIONS IS NOT NULL;

,Count
0,841


In [16]:
%%sql
SELECT *
FROM TFL
WHERE (MONTH = 2 AND DAYS > 29) OR (MONTH IN (4, 6, 9, 11) AND DAYS > 30);

,month,year,days,report_date,journey_type,journeys_millions


In [17]:
%%sql
SELECT *
FROM TFL
WHERE (MONTH = 2 AND DAYS > 29) OR (MONTH IN (4, 6, 9, 11) AND DAYS > 30);

,month,year,days,report_date,journey_type,journeys_millions


In [18]:
%%sql
SELECT JOURNEY_TYPE, AVG(JOURNEYS_MILLIONS) AS avg_journeys, MAX(JOURNEYS_MILLIONS) AS max_journeys, MIN(JOURNEYS_MILLIONS) AS min_journeys
FROM TFL
GROUP BY JOURNEY_TYPE;

,journey_type,avg_journeys,max_journeys,min_journeys
0,Bus,159.648679,203.398289,30.223736
1,Overground,11.186884,17.820632,0.999693
2,Underground & DLR,96.285042,128.155995,6.950757
3,Tram,2.017243,2.638906,0.440934
4,Emirates Airline,0.113052,0.534218,0.000169
5,TfL Rail,4.329615,15.223201,0.594615


In [31]:
%%sql
WITH Stats AS (
    SELECT 
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY JOURNEYS_MILLIONS) AS median,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY JOURNEYS_MILLIONS) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY JOURNEYS_MILLIONS) AS q3
    FROM TFL
),
Filtered AS (
    SELECT *
    FROM TFL t
    JOIN Stats s ON 1=1
    WHERE t.JOURNEYS_MILLIONS IS NOT NULL AND
          t.JOURNEYS_MILLIONS BETWEEN s.median - 1.5 * (s.q3 - s.q1) AND s.median + 1.5 * (s.q3 - s.q1)
)
SELECT *
FROM Filtered;

,month,year,days,report_date,journey_type,journeys_millions,median,q1,q3
0,1,2010,31,06/30/2010,Underground & DLR,96.836391,9.780669,2.133444,105.064674
1,2,2010,28,06/30/2010,Underground & DLR,90.330504,9.780669,2.133444,105.064674
2,3,2010,31,06/30/2010,Underground & DLR,90.038014,9.780669,2.133444,105.064674
3,4,2010,30,09/30/2010,Underground & DLR,92.544093,9.780669,2.133444,105.064674
4,5,2010,31,09/30/2010,Underground & DLR,88.662911,9.780669,2.133444,105.064674
...,...,...,...,...,...,...,...,...,...
737,8,2022,31,12/31/2022,TfL Rail,12.515850,9.780669,2.133444,105.064674
738,9,2022,30,12/31/2022,TfL Rail,15.223201,9.780669,2.133444,105.064674
739,10,2022,31,03/31/2023,TfL Rail,8.875033,9.780669,2.133444,105.064674
740,11,2022,30,03/31/2023,TfL Rail,13.404840,9.780669,2.133444,105.064674
